In [141]:
from imblearn.over_sampling import SMOTE
import sklearn as sk
import random as rnd

In [135]:
%run -i "data_preproc.py"

Reading file: orbis_active_be.xlsx ...
Reading file: orbis_active_de.xlsx ...
Reading file: orbis_active_es.xlsx ...
Reading file: orbis_active_fin.xlsx ...
Reading file: orbis_active_fra.xlsx ...
Reading file: orbis_active_it.xlsx ...
Reading file: orbis_active_no.xlsx ...
Reading file: orbis_active_rest.xlsx ...
Reading file: orbis_active_se.xlsx ...
Reading file: orbis_default.xlsx ...


In [136]:
df['country'].groupby([df['country']]).count().sort_values(ascending=False)

country
IT    35458
SE    34660
ES    33540
FR    33117
DE    33100
DK    32246
FI    24967
NO    21206
NL     4676
AT     3080
IS     2364
BE      640
CY       42
LU        1
Name: country, dtype: int64

In [137]:
(df.loc[:, ~df.columns.str.contains('ratio|ch|sector_')].isna().sum() / df.shape[0]).sort_values(ascending=False) # NAs share among original variables

revenue_lag     0.198200
revenue         0.171048
lt_debt         0.079032
cash            0.045801
pnl_lag         0.032853
ebit_lag        0.032235
assets_lag      0.020367
capital         0.010818
sector          0.005318
ebit            0.000120
fixed_assets    0.000012
assets          0.000004
assets_log      0.000004
CY              0.000000
FR              0.000000
IS              0.000000
SE              0.000000
AT              0.000000
BE              0.000000
NL              0.000000
NO              0.000000
IT              0.000000
Inactive        0.000000
FI              0.000000
ES              0.000000
DE              0.000000
DK              0.000000
country         0.000000
curr_liab       0.000000
curr_assets     0.000000
pnl             0.000000
last_year       0.000000
LU              0.000000
dtype: float64

In [140]:
split_share = 0.8

rnd.seed(1)
train_id = rnd.sample(range(df.shape[0]), round(df.shape[0] * split_share))

train_df = df.drop(['country', 'last_year', 'sector'], axis=1).loc[train_id]
test_df  = df.drop(['country', 'last_year', 'sector'], axis=1).loc[~np.isin(list(range(df.shape[0])), train_id)]

In [212]:

# univariate imputing with train dataset median of given variable. 
for variable in train_df.loc[:, train_df.apply(lambda x: any(x.isna()))].columns:
    train_df.loc[train_df[variable].isna(), variable] = train_df[variable].median()
    test_df.loc[test_df[variable].isna(), variable] = test_df[variable].median()

# idea to implement: multivariate LM imputing based on no NAs variable

In [215]:
X_train_sm, y_train_sm = SMOTE(random_state=1).fit_resample(train_df.loc[:,~np.isin(train_df.columns, 'Inactive')], train_df['Inactive'])


In [270]:
rnd.seed(1)
os_indexes = rnd.choices(list(train_df[train_df.Inactive == 1].index), k = np.sum(train_df.Inactive == 0) - np.sum(train_df.Inactive == 1))

train_df_os = pd.concat([train_df.loc[os_indexes,:], train_df])

,Inactive,pnl,pnl_lag,assets,assets_lag,fixed_assets,curr_assets,capital,lt_debt,curr_liab,...,FR,IT,NO,NL,AT,IS,CY,SE,BE,LU
258377,1,118,-157,3569,3223,785,2784,67,0,2088,...,0,0,0,0,0,0,0,0,0,0
256083,1,65,3,1337,674,76,1260,102,0,1090,...,0,1,0,0,0,0,0,0,0,0
254294,1,4,-79,400,560,10,391,20,0,381,...,0,0,0,0,0,0,0,1,0,0
253389,1,-529,-458,2111,2222,1107,1004,23,194,577,...,0,0,0,0,0,0,0,1,0,0
252613,1,569,265,3168,2236,188,2980,5,0,2000,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224917,0,247,182,2076,1962,18,2057,13,0,1301,...,0,0,0,0,0,0,0,1,0,0
46930,0,380,174,16908,17168,15491,1417,150,7557,752,...,0,0,0,0,0,0,0,0,0,0
106843,0,10,65.0,70425,1606.0,22623,47802,1250,0,56214,...,0,0,0,0,0,0,0,0,0,0
24491,0,52,2,369,215,48,321,7,0,288,...,0,0,0,0,0,0,0,0,0,0
